In [1]:
#导入包
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [2]:
#分成数据集和测试集
housing = fetch_california_housing()
# print(housing.data.shape)

x_train_all, x_test_raw, y_train_all, y_test = train_test_split(
    housing.data, housing.target, random_state = 7)
x_train_raw, x_valid_raw, y_train, y_valid = train_test_split(
    x_train_all, y_train_all, random_state = 11)

#对数据进行标准化处理
sca = StandardScaler()
x_train = sca.fit_transform(x_train_raw)
x_valid = sca.transform(x_valid_raw)
x_test = sca.transform(x_test_raw)

# 自定义Denselayer、
    cell_one：通过子类继承的方法来实现自定义Dense_layer
    cell_two：通过tf.keras.Lambda(lambda ～～～)来实现简单的layer

## method_one:通过子类继承的方法来实现自定义dense_layer
    (1)__init__()函数：用来继承父类keras.layer.Layer。
    (2)build()函数：用来构建所需要的参数
    (3)call()函数：完成正向计算

In [ ]:
# customized dense layer.
class CustomizedDenseLayer(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        self.units = units
        self.activation = keras.layers.Activation(activation)
        super(CustomizedDenseLayer, self).__init__(**kwargs)
    
    def build(self, input_shape):
        """构建所需要的参数"""
        # x * w + b. input_shape:[None, a] w:[a,b]output_shape: [None, b]
        self.kernel = self.add_weight(name = 'kernel',
                                      shape = (input_shape[1], self.units),
                                      initializer = 'uniform',
                                      trainable = True)
        self.bias = self.add_weight(name = 'bias',
                                    shape = (self.units, ),
                                    initializer = 'zeros',
                                    trainable = True)
        super(CustomizedDenseLayer, self).build(input_shape)
    
    def call(self, x):
        """完成正向计算"""
        return self.activation(x @ self.kernel + self.bias)
    
'''
kelnel代表w，其参数：
name:参数的名称

shape:设输入数据为[num，a]，该层的神经元数为units，则根据x * w + b,则w的shape应该是[a, units],a为x每一个样本输入数据的个数，
units代表该层神经元的数量，也就是说该层的每个神经元都有其参数，每个神经元的参数数量与一个样本的数据量相同。
其实shape也可以不指定，因为tf可以自行推断输入数据的形状

initializer：指定参数的初始化分布，uniform代表均匀分布

trainable：指定是否为可训练参数
'''






## method_two：通过tf.keras.Lambda(lambda ~）函数
    不过这种方法应该只能实现简单的单神经元的layer

In [ ]:
# tf.nn.softplus : log(1+e^x)
customized_softplus = keras.layers.Lambda(lambda x : tf.nn.softplus(x))
print(customized_softplus([-10., -5., 0., 5., 10.]))

In [ ]:
#构建模型
model = keras.models.Sequential()

model.add(keras.layers.Dense(8,activation='relu', input_shape = x_train.shape[1:]))
model.add(keras.layers.Dense(1))

model.compile(loss = 'mean_squared_error', optimizer = 'adam')  
#因为是回归模型，所以判断模型好坏的标准就是均方差的大小，而不需要加上评价标准 metrics = ['accuracy']

history = model.fit(x_train, y_train,
          validation_data = (x_valid, y_valid),
          callbacks = [keras.callbacks.EarlyStopping(patience = 5, min_delta = 1e-3)],
          epochs = 20)


In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 2)
    plt.show()
plot_learning_curves(history)


In [ ]:
model.evaluate(x_train, y_train, verbose = False)